In [1]:
import pandas as pd
import requests
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

## Household Income Data
* Load Household Income data from url into a DataFrame
* Clean data - filter for latest year of data (2018)
* Save to csv

In [2]:
# Output File (CSV)
output_data_file = "output_data/household_income.csv"

In [3]:
# Save config information
household_url = "https://datausa.io/api/data?Geography=16000US4260000:tracts&measure=Household%20Income%20by%20Race,Household%20Income%20by%20Race%20Moe&Race=0"

# Build query URL
household = f"{household_url}"
household

'https://datausa.io/api/data?Geography=16000US4260000:tracts&measure=Household%20Income%20by%20Race,Household%20Income%20by%20Race%20Moe&Race=0'

In [4]:
# Retrieve json data using requests
response = requests.get(household).json()
response

{'data': [{'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 102448,
   'Household Income by Race Moe': 16740,
   'Geography': 'Census Tract 1, Philadelphia County, PA',
   'ID Geography': '14000US42101000100'},
  {'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 47243,
   'Household Income by Race Moe': 14802.999999999998,
   'Geography': 'Census Tract 2, Philadelphia County, PA',
   'ID Geography': '14000US42101000200'},
  {'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 91354,
   'Household Income by Race Moe': 18636,
   'Geography': 'Census Tract 3, Philadelphia County, PA',
   'ID Geography': '14000US42101000300'},
  {'ID Year': 2018,
   'Year': '2018',
   'ID Race': 0,
   'Race': 'Total',
   'Household Income by Race': 62821,
   'Household Income by Race Moe': 10734,
   'Geography': 'Census Tract 4.01, Philadelphia Co

In [5]:
# Create a variable to hold json output
household_data = response['data']
household_data

[{'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 102448,
  'Household Income by Race Moe': 16740,
  'Geography': 'Census Tract 1, Philadelphia County, PA',
  'ID Geography': '14000US42101000100'},
 {'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 47243,
  'Household Income by Race Moe': 14802.999999999998,
  'Geography': 'Census Tract 2, Philadelphia County, PA',
  'ID Geography': '14000US42101000200'},
 {'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 91354,
  'Household Income by Race Moe': 18636,
  'Geography': 'Census Tract 3, Philadelphia County, PA',
  'ID Geography': '14000US42101000300'},
 {'ID Year': 2018,
  'Year': '2018',
  'ID Race': 0,
  'Race': 'Total',
  'Household Income by Race': 62821,
  'Household Income by Race Moe': 10734,
  'Geography': 'Census Tract 4.01, Philadelphia County, PA',
  'ID Geography': '14000US42

In [6]:
# Set empty lists to hold data needed 

ID_Geography = []
Household_Income_by_Race = []
ID_Year = []


# Loop through each salary info
for info in household_data:
    ID_Geography.append(info['ID Geography'])
    Household_Income_by_Race.append(info['Household Income by Race'])
    ID_Year.append(info['ID Year'])

In [7]:
#create DataFrame with data
household_summary = {"ID_Geography":ID_Geography, "Household_Income": Household_Income_by_Race, "ID_Year": ID_Year}
household_df = pd.DataFrame(household_summary)
household_df

,ID_Geography,Household_Income,ID_Year
0,14000US42101000100,102448,2018
1,14000US42101000200,47243,2018
2,14000US42101000300,91354,2018
3,14000US42101000401,62821,2018
4,14000US42101000402,78648,2018
...,...,...,...
2243,14000US42101038900,39886,2013
2244,14000US42101039000,30081,2013
2245,14000US42101980000,39861,2013
2246,14000US42101980100,66250,2013


In [8]:
# Split 'ID_Geography' into two columns and store GEOID in new column
household_df [["First", "GEOID"]] = household_df.ID_Geography.str.split("S", expand = True)
household_df.drop(columns = ["First", "ID_Geography"], inplace = True)

# Filter for latest year of data - 2018
household_summary_df = household_df[household_df['ID_Year'] == 2018]
household_summary_df

,Household_Income,ID_Year,GEOID
0,102448,2018,42101000100
1,47243,2018,42101000200
2,91354,2018,42101000300
3,62821,2018,42101000401
4,78648,2018,42101000402
...,...,...,...
369,75313,2018,42101038800
370,38442,2018,42101038900
371,29034,2018,42101039000
372,62000,2018,42101980000


In [9]:
# Save to csv
household_summary_df.to_csv("household_income_summary.csv", encoding="utf-8", index=False)

## Census Tract Data
* Load census tracts information from url into DataFrame
* Extract each coordinate in Tract Coordinates and store in a separate row
* Save to csv

In [34]:
# Create a variable for the url
census_url = "https://services.arcgis.com/fLeGjb7u4uXqeF9q/arcgis/rest/services/Census_Tracts_2010/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

In [35]:
# Retrieve json data using requests
response = requests.get(census_url).json()

In [36]:
# Create a variable to hold json output
census_tracts_data = response['features']
census_tracts_data

[{'type': 'Feature',
  'id': 1,
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-75.2292652278066, 39.9605446656843],
     [-75.2297874956556, 39.9606084375867],
     [-75.2302498485, 39.9606653834397],
     [-75.230804767063, 39.9607313733177],
     [-75.2311897934545, 39.9607760640567],
     [-75.2318765350236, 39.9608591333805],
     [-75.2328598242113, 39.9609850764494],
     [-75.2335557092359, 39.9610750702774],
     [-75.2341439263989, 39.9611452556569],
     [-75.2348718979056, 39.961240293228],
     [-75.2368108342578, 39.961480097949],
     [-75.2366791596268, 39.9621026264806],
     [-75.2366426818682, 39.962294651098],
     [-75.2365901520543, 39.9625285843249],
     [-75.236483638182, 39.9630562034783],
     [-75.2363149009782, 39.9638135023702],
     [-75.2361656005889, 39.9645743506832],
     [-75.2360056992624, 39.9653348999137],
     [-75.2358447411698, 39.966094703231],
     [-75.234856877156, 39.965971761409],
     [-75.2339060332792, 39.9658560631339],
     [

In [37]:
# Create empty lists to hold information
GEOID = []
Lat = []
Lng = []
Geometry = []


# Loop through each record in json data and append data to relevant list
for info in census_tracts_data:
    GEOID.append(info['properties']['GEOID10'])
    Lat.append(info['properties']['INTPTLAT10'])
    Lng.append(info['properties']['INTPTLON10'])
    Geometry.append(info['geometry']['coordinates'][0])

In [46]:
# Create a DataFrame to store information
census_tracts_df = pd.DataFrame({"geoid": GEOID, "latitude": Lat,\
                                         "longitude": Lng, "Tract Coordinates": Geometry})

census_tracts_df.head()

,geoid,latitude,longitude,Tract Coordinates
0,42101009400,+39.9632709,-075.2322437,"[[-75.2292652278066, 39.9605446656843], [-75.2..."
1,42101009500,+39.9658709,-075.2379140,"[[-75.2353591940378, 39.9685190282368], [-75.2..."
2,42101009600,+39.9655396,-075.2435075,"[[-75.2434277968255, 39.9622970641896], [-75.2..."
3,42101013800,+39.9764504,-075.1771771,"[[-75.1734078067174, 39.977792764172], [-75.17..."
4,42101013900,+39.9750563,-075.1711846,"[[-75.1731295290552, 39.9777597701621], [-75.1..."


In [47]:
# Expand Tract Coordinates such that each point is a separate row
expanded_df = pd.DataFrame()
for row in census_tracts_df.index:
    col = census_tracts_df.loc[row, 'Tract Coordinates']
    N = len(col)
    expanded_df = expanded_df.append(pd.DataFrame(
        [[c, census_tracts_df.loc[row, 'geoid'], census_tracts_df.loc[row, 'latitude'],\
          census_tracts_df.loc[row, 'longitude']] for c in col],
        index=[row] * N,
        columns = ['Tract Coordinates', 'geoid', 'latitude', 'longitude']))
expanded_df.head()

,Tract Coordinates,geoid,latitude,longitude
0,"[-75.2292652278066, 39.9605446656843]",42101009400,+39.9632709,-075.2322437
0,"[-75.2297874956556, 39.9606084375867]",42101009400,+39.9632709,-075.2322437
0,"[-75.2302498485, 39.9606653834397]",42101009400,+39.9632709,-075.2322437
0,"[-75.230804767063, 39.9607313733177]",42101009400,+39.9632709,-075.2322437
0,"[-75.2311897934545, 39.9607760640567]",42101009400,+39.9632709,-075.2322437


In [49]:
# Create a DataFrame that splits Tract Coordinates into separate points
census_summary_df = expanded_df.copy()
census_summary_df['coordinate_0'] = [i[0] for i in census_summary_df['Tract Coordinates']]
census_summary_df['coordinate_1'] = [i[1] for i in census_summary_df['Tract Coordinates']]

# Drop unused column
# Reset index and rename columns
census_summary_df.drop(columns = ['Tract Coordinates'], inplace = True)
census_summary_df.reset_index(inplace = True)
census_summary_df.rename(columns={"index":"recordnum"}, inplace = True)
census_summary_df.index.names = ['index']
census_summary_df.head()

,recordnum,geoid,latitude,longitude,coordinate_0,coordinate_1
index,,,,,,
0,0,42101009400,+39.9632709,-075.2322437,-75.2293,39.9605
1,0,42101009400,+39.9632709,-075.2322437,-75.2298,39.9606
2,0,42101009400,+39.9632709,-075.2322437,-75.2302,39.9607
3,0,42101009400,+39.9632709,-075.2322437,-75.2308,39.9607
4,0,42101009400,+39.9632709,-075.2322437,-75.2312,39.9608


In [50]:
census_tracts_summary_df = census_summary_df.applymap(str)

In [51]:
# Save information in a csv file
census_tracts_summary_df.to_csv("census_tracts_summary.csv", encoding="utf-8", index=False)

# Schools Data

In [ ]:
schools_data = pd.read_csv("schools_clean.csv")
schools_data.head()

In [ ]:
schools_data.rename(columns = {"Publication Name" : "School_Name", 
                               "Abbreviated Name": "Abbreviated_Name",
                               "School Level": "School_Level",
                               "Admission Type": "Admission_Type",
                               "Current Grade Span Served": "Grade_Level",
                               "GPS Location":"Geolocation",
                               "Street Address": "Address"
                              }, inplace=True)

In [ ]:
schools_data.head()

## Join Data to create new DataFrame

In [52]:
# Create a copy of the census_tracts_df to get a concise record of census data
tracts_concise_df = census_tracts_df.copy()
tracts_concise_df.drop(columns=['Tract Coordinates'], inplace = True)
tracts_concise_df.head()

,geoid,latitude,longitude
0,42101009400,+39.9632709,-075.2322437
1,42101009500,+39.9658709,-075.2379140
2,42101009600,+39.9655396,-075.2435075
3,42101013800,+39.9764504,-075.1771771
4,42101013900,+39.9750563,-075.1711846


In [53]:
# Merge census data and household data
merged_df = household_summary_df.merge(tracts_concise_df,\
                                       how = "right" , left_on = "GEOID",\
                                       right_on = "geoid")
merged_df.drop(columns = ['GEOID'], inplace = True)
merged_df.index.names = ['index']
merged_df

,Household_Income,ID_Year,geoid,latitude,longitude
index,,,,,
0,102448.0,2018.0,42101000100,+39.9523827,-075.1466628
1,47243.0,2018.0,42101000200,+39.9553999,-075.1569775
2,91354.0,2018.0,42101000300,+39.9568780,-075.1716655
3,62821.0,2018.0,42101000401,+39.9541871,-075.1758082
4,78648.0,2018.0,42101000402,+39.9532973,-075.1686952
...,...,...,...,...,...
379,NaN,NaN,42101980600,+39.9055378,-075.1707151
380,NaN,NaN,42101980800,+39.9714517,-075.2621770
381,NaN,NaN,42101980500,+40.0177581,-075.1441820


In [54]:
#create a csv

merged_df.to_csv("merged_df.csv", encoding="utf-8", index=False)


## Transfering data to SQLite Database

In [55]:
# Create the database engine
engine = create_engine('sqlite:///philly.sqlite', echo=False)
sqlite_connection = engine.connect()

### Household Data

In [ ]:
# engine.execute('DROP TABLE "censustracts"' )
# engine.execute('DROP TABLE "household"' )
# engine.execute('DROP TABLE "census_household"' )

In [ ]:
# Add primary key to "merged" table that contains all cleaned data
sqlite_connection.execute('DROP TABLE "household"' )
sqlite_connection.execute('CREATE TABLE "household" ( "index" INTEGER PRIMARY KEY, "GEOID" TEXT, "Household_Income" INTEGER, "ID_Year" INTEGER )' )

In [ ]:
sqlite_table = "household"
household_summary_df.to_sql(sqlite_table, sqlite_connection, if_exists='append', index=True, )

In [ ]:
# Check what data exists in the censustracts table
engine.execute("SELECT * from household;").fetchall()

### Census Tracts Data

In [56]:
# Define censustracts table
class Census(Base):
    __tablename__ = 'censustracts'
    index = Column(Integer, primary_key = True)
    recordnum = Column(String)
    geoid = Column(String)
    latitude = Column(String)
    longitude = Column(String)
    coordinate_0 = Column(String)
    coordinate_1 = Column(String)

In [57]:
# Check table information - as it exists in python
Base.metadata.tables

immutabledict({'censustracts': Table('censustracts', MetaData(bind=None), Column('index', Integer(), table=<censustracts>, primary_key=True, nullable=False), Column('recordnum', String(), table=<censustracts>), Column('geoid', String(), table=<censustracts>), Column('latitude', String(), table=<censustracts>), Column('longitude', String(), table=<censustracts>), Column('coordinate_0', String(), table=<censustracts>), Column('coordinate_1', String(), table=<censustracts>), schema=None)})

In [58]:
# Create the tables in the database
Base.metadata.create_all(engine)

In [59]:
# Check what data exists in the censustracts table
engine.execute("SELECT * from censustracts;").fetchall()

[]

In [60]:
# Populate table with contents from the census_tracts_summary_df
census_tracts_summary_df.to_sql("censustracts", con=engine, if_exists = 'append', index = True)

In [61]:
# Check what data exists in the censustracts table
engine.execute("SELECT * from censustracts;").fetchall()

[(0, '0', '42101009400', '+39.9632709', -75.2322437, -75.2292652278066, 39.9605446656843),
 (1, '0', '42101009400', '+39.9632709', -75.2322437, -75.2297874956556, 39.9606084375867),
 (2, '0', '42101009400', '+39.9632709', -75.2322437, -75.2302498485, 39.9606653834397),
 (3, '0', '42101009400', '+39.9632709', -75.2322437, -75.230804767063, 39.9607313733177),
 (4, '0', '42101009400', '+39.9632709', -75.2322437, -75.2311897934545, 39.9607760640567),
 (5, '0', '42101009400', '+39.9632709', -75.2322437, -75.2318765350236, 39.9608591333805),
 (6, '0', '42101009400', '+39.9632709', -75.2322437, -75.2328598242113, 39.9609850764494),
 (7, '0', '42101009400', '+39.9632709', -75.2322437, -75.2335557092359, 39.9610750702774),
 (8, '0', '42101009400', '+39.9632709', -75.2322437, -75.2341439263989, 39.9611452556569),
 (9, '0', '42101009400', '+39.9632709', -75.2322437, -75.2348718979056, 39.961240293228),
 (10, '0', '42101009400', '+39.9632709', -75.2322437, -75.2368108342578, 39.961480097949),
 (11

### Census and Household Merged Data

In [62]:
# Define censustracts table
class Census_Household(Base):
    __tablename__ = 'census_household'
    index = Column(Integer, primary_key = True)
    Household_Income = Column(Integer)
    ID_Year = Column(Integer)
    geoid = Column(String)
    latitude = Column(String)
    longitude = Column(String)

In [63]:
# Check table information - as it exists in python
Base.metadata.tables

immutabledict({'censustracts': Table('censustracts', MetaData(bind=None), Column('index', Integer(), table=<censustracts>, primary_key=True, nullable=False), Column('recordnum', String(), table=<censustracts>), Column('geoid', String(), table=<censustracts>), Column('latitude', String(), table=<censustracts>), Column('longitude', String(), table=<censustracts>), Column('coordinate_0', String(), table=<censustracts>), Column('coordinate_1', String(), table=<censustracts>), schema=None), 'census_household': Table('census_household', MetaData(bind=None), Column('index', Integer(), table=<census_household>, primary_key=True, nullable=False), Column('Household_Income', Integer(), table=<census_household>), Column('ID_Year', Integer(), table=<census_household>), Column('geoid', String(), table=<census_household>), Column('latitude', String(), table=<census_household>), Column('longitude', String(), table=<census_household>), schema=None)})

In [64]:
# Create the tables in the database
Base.metadata.create_all(engine)

In [65]:
# Populate table with contents from the merged_df
merged_df.to_sql('census_household', con=engine, if_exists = 'append', index = True)

In [66]:
# Check what data exists in the censustracts table
engine.execute("SELECT * from census_household;").fetchall()

[(0, 102448, 2018, '42101000100', '+39.9523827', '-075.1466628'),
 (1, 47243, 2018, '42101000200', '+39.9553999', '-075.1569775'),
 (2, 91354, 2018, '42101000300', '+39.9568780', '-075.1716655'),
 (3, 62821, 2018, '42101000401', '+39.9541871', '-075.1758082'),
 (4, 78648, 2018, '42101000402', '+39.9532973', '-075.1686952'),
 (5, 43702, 2018, '42101000500', '+39.9519534', '-075.1581776'),
 (6, 82833, 2018, '42101000600', '+39.9493845', '-075.1583354'),
 (7, 55250, 2018, '42101000700', '+39.9510463', '-075.1729143'),
 (8, 125822, 2018, '42101000801', '+39.9497374', '-075.1804054'),
 (9, 59560, 2018, '42101000803', '+39.9493748', '-075.1742489'),
 (10, 92281, 2018, '42101000804', '+39.9485646', '-075.1677563'),
 (11, 44444, 2018, '42101000901', '+39.9478866', '-075.1623004'),
 (12, 59174, 2018, '42101000902', '+39.9471596', '-075.1566389'),
 (13, 104250, 2018, '42101001001', '+39.9456722', '-075.1516020'),
 (14, 122358, 2018, '42101001002', '+39.9450045', '-075.1466193'),
 (15, 65000, 201

### Schools Data

In [ ]:
sqlite_connection.execute('DROP TABLE "philly_schools"')
sqlite_connection.execute('CREATE TABLE "philly_schools" ("index" INTEGER PRIMARY KEY, "School Name" TEXT, "Abbreviated Name" TEXT, "School Level" TEXT, "Admission Type" TEXT, "Current Grade Span Served" TEXT, "Geolocation" TEXT, "Street Address" TEXT, "Website" TEXT)')

In [ ]:
sqlite_table = "philly_schools"
schools_data.to_sql(sqlite_table, sqlite_connection, if_exists = "replace", index =True)

In [ ]:
sqlite_connection.close()